# Asset Selection

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [3]:
# import bs4 as bs

# response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# soup = bs.BeautifulSoup(response.text)
# table = soup.find('table', {'class': 'wikitable sortable'})

# tickers = []
# for row in table.findAll('tr')[1:]:
#     ticker = row.findAll('td')[0].text.replace('\n', '')
#     if '.' in ticker:
#         ticker = ticker.replace('.', '-')
#     tickers.append(ticker)

# with open('../data/sp500tickers.pickle', 'wb') as f:
#     pk.dump(tickers, f)

# print('---------------')
# print(f'Ticker Amount: {len(tickers)}')
# print('---------------')

# with open('../data/sp500tickers.pickle', 'rb') as f:
#     sp500 = pk.load(f)

# with open('../../../alphavantage_token.TXT', 'rb') as f:
#     TOKEN = f.read().decode('utf-8')

# for ticker in sp500[501:]:
#     url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={TOKEN}'
#     r = requests.get(url)
#     data.loc[ticker] = r.json()

# data.head()

---------------
Ticker Amount: 503
---------------


In [3]:
d = {'Symbol': 'MMM', 'AssetType': 'Common Stock', 'Name': '3M Company',
     'Description': 'ood, a suburb of Saint Paul, Minnesota.',
     'CIK': '66740', 'Exchange': 'NYSE', 'Currency': 'USD',
     'Country': 'USA', 'Sector': 'LIFE SCIENCES',
     'Industry': 'SURGICAL & MEDICAL INSTRUMENTS & APPARATUS',
     'Address': '3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US',
     'FiscalYearEnd': 'December', 'LatestQuarter': '2024-03-31',
     'MarketCapitalization': '57903698000', 'EBITDA': '8146000000',
     'PERatio': 'None', 'PEGRatio': '1.905', 
      
     'BookValue': '8.79',
     'DividendPerShare': '6.01', 'DividendYield': '0.0268',
     'EPS': '-12.73', 'RevenuePerShareTTM': '58.89',
     'ProfitMargin': '-0.216', 'OperatingMarginTTM': '0.217',
     'ReturnOnAssetsTTM': '0.0758', 
     
     'ReturnOnEquityTTM': '-0.693',
     'RevenueTTM': '32653001000', 'GrossProfitTTM': '15000000000',
     'DilutedEPSTTM': '-12.73', 'QuarterlyEarningsGrowthYOY': '-0.051',
     'QuarterlyRevenueGrowthYOY': '-0.003', 'AnalystTargetPrice': '110',
     'AnalystRatingStrongBuy': '0', 'AnalystRatingBuy': '5',
     'AnalystRatingHold': '11', 'AnalystRatingSell': '2',
     'AnalystRatingStrongSell': '0', 'TrailingPE': '-',
     'ForwardPE': '10.6', 'PriceToSalesRatioTTM': '1.546',
     'PriceToBookRatio': '10.51', 'EVToRevenue': '1.879',
     'EVToEBITDA': '-9.08', 'Beta': '1.001', '52WeekHigh': '105.31',
     '52WeekLow': '68.63', '50DayMovingAverage': '101.16',
     '200DayMovingAverage': '97.91', 'SharesOutstanding': '553361000',
     'DividendDate': '2024-06-12', 'ExDividendDate': '2024-05-23'}
d.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold', 'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [4]:
data = pd.read_csv('../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740.0,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,"3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US",...,1.879,-9.08,1.001,105.52,68.63,101.50,98.13,5.533610e+08,2024-06-12,2024-05-23
AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142.0,NYSE,USD,USA,MANUFACTURING,HOUSEHOLD APPLIANCES,"11270 WEST PARK PLACE, MILWAUKEE, WI, US",...,3.248,15.19,1.141,92.44,63.34,83.92,81.37,1.207840e+08,2024-08-15,2024-07-31
ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"100 ABBOTT PARK ROAD, ABBOTT PARK, IL, US",...,4.928,18.75,0.718,120.40,88.33,103.84,106.60,1.739630e+09,2024-08-15,2024-07-15
ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,"1 NORTH WAUKEGAN ROAD, NORTH CHICAGO, IL, US",...,6.15,19.45,0.623,179.55,132.11,166.45,162.53,1.765870e+09,2024-08-15,2024-07-15
ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373.0,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","1 GRAND CANAL SQUARE, GRAND CANAL HARBOUR, DUB...",...,3.041,18.21,1.227,384.33,277.47,301.44,329.36,6.263840e+08,2024-08-15,2024-07-11


In [7]:
data.drop(['AssetType', 'CIK', 'Description', 'Currency', 'Country', 'Address'], inplace=True, axis=1)

In [8]:
data['ReturnOnAssetsTTM'].replace('None', np.nan, inplace=True)
data['Beta'].replace('None', np.nan, inplace=True)
data['PEGRatio'].replace('None', np.nan, inplace=True)
data['PERatio'].replace('None', np.nan, inplace=True)
data['BookValue'].replace('None', np.nan, inplace=True)
data['DividendPerShare'].replace('None', np.nan, inplace=True)
data['DividendYield'].replace('None', np.nan, inplace=True)
data['EPS'].replace('None', np.nan, inplace=True)
data['RevenuePerShareTTM'].replace('None', np.nan, inplace=True)
data['ProfitMargin'].replace('None', np.nan, inplace=True)
data['OperatingMarginTTM'].replace('None', np.nan, inplace=True)
data['MarketCapitalization'].replace('None', np.nan, inplace=True)
data['EBITDA'].replace('None', np.nan, inplace=True)

In [9]:
data['MarketCapitalization'] = data['MarketCapitalization'].astype(float)
data['Beta'] = data['Beta'].astype(float)
data['PEGRatio'] = data['PEGRatio'].astype(float)
data['PERatio'] = data['PERatio'].astype(float)
data['BookValue'] = data['BookValue'].astype(float)
data['DividendPerShare'] = data['BookValue'].astype(float)
data['DividendYield'] = data['DividendYield'].astype(float)
data['EPS'] = data['EPS'].astype(float)
data['RevenuePerShareTTM'] = data['RevenuePerShareTTM'].astype(float)
data['ProfitMargin'] = data['ProfitMargin'].astype(float)
data['OperatingMarginTTM'] = data['OperatingMarginTTM'].astype(float)
data['ReturnOnAssetsTTM'] = data['ReturnOnAssetsTTM'].astype(float)
data['EBITDA'] = data['EBITDA'].astype(float)

## Multiplicators

In [10]:
clmns = ['Name', 'MarketCapitalization', 'Sector', 'Industry', 'PERatio', 'PEGRatio', 'Beta', 'DividendPerShare', 'DividendYield']

In [11]:
cat = data.copy()

### Market Capitalization

In [12]:
data['MarketCapitalization'].describe()

count    4.330000e+02
mean     1.002335e+11
std      2.985303e+11
min      7.004214e+09
25%      1.867546e+10
50%      3.621605e+10
75%      7.378835e+10
max      3.450326e+12
Name: MarketCapitalization, dtype: float64

In [13]:
market_cap = data['MarketCapitalization']
bins = pd.qcut(market_cap, 3)
results = (pd.Series(market_cap).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,MarketCapitalization,count,min,max
0,"(7004213999.999, 22302878000.0]",145,7.004214e+09,2.230288e+10
1,"(22302878000.0, 53696979000.0]",144,2.233913e+10,5.369698e+10
2,"(53696979000.0, 3450325697000.0]",144,5.503403e+10,3.450326e+12


In [14]:
cat['market_cap_bins'] = bins
cat.groupby(['Sector', 'market_cap_bins']).count()['Name'].to_frame()

Name
Sector                     market_cap_bins                       
ENERGY & TRANSPORTATION    (7004213999.999, 22302878000.0]     18
                           (22302878000.0, 53696979000.0]      26
                           (53696979000.0, 3450325697000.0]    16
FINANCE                    (7004213999.999, 22302878000.0]     18
                           (22302878000.0, 53696979000.0]      16
                           (53696979000.0, 3450325697000.0]    22
LIFE SCIENCES              (7004213999.999, 22302878000.0]     21
                           (22302878000.0, 53696979000.0]      22
                           (53696979000.0, 3450325697000.0]    26
MANUFACTURING              (7004213999.999, 22302878000.0]     36
                           (22302878000.0, 53696979000.0]      26
                           (53696979000.0, 3450325697000.0]    28
REAL ESTATE & CONSTRUCTION (7004213999.999, 22302878000.0]     14
                           (22302878000.0, 53696979000.0]      16
                           (53696979000.0, 3450325697000.0]     6
TECHNOLOGY                 (7004213999.999, 22302878000.0]     18
                           (22302878000.0, 53696979000.0]      14
                           (53696979000.0, 3450325697000.0]    27
TRADE & SERVICES           (7004213999.999, 22302878000.0]     20
                           (22302878000.0, 53696979000.0]      24
                           (53696979000.0, 3450325697000.0]    19

In [15]:
data.sort_values(by='MarketCapitalization', ascending=False).head(10)

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,NASDAQ,TECHNOLOGY,ELECTRONIC COMPUTERS,September,2024-03-31,3.450326e+12,129629004000,34.94,2.351,...,9.11,26.13,1.244,237.23,163.86,207.76,187.9,15334100000.0,2024-05-16,2024-05-10
MSFT,Microsoft Corporation,NASDAQ,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,June,2024-06-30,3.036322e+12,129433002000,34.59,2.200,...,12.35,22.76,0.894,468.35,307.7,438.57,403.82,7433040000.0,2024-09-12,2024-08-15
GOOG,Alphabet Inc Class C,NASDAQ,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",December,2024-03-31,2.254508e+12,109723001000.0,28.08,1.659,...,6.14,19.27,1.046,193.31,121.32,180.07,152.99,5617000000.0,2024-06-17,2024-06-10
GOOGL,Alphabet Inc Class A,NASDAQ,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",December,2024-03-31,2.254507e+12,109723001000.0,27.86,1.669,...,6.99,20.42,1.046,191.75,120.07,179.13,152.33,5874000000.0,2024-06-17,2024-06-10
AMZN,Amazon.com Inc,NASDAQ,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,December,2024-03-31,1.899725e+12,96609001000.0,50.99,1.987,...,3.268,19.57,1.155,201.2,118.35,187.51,167.96,10406600000.0,None,None
META,Meta Platforms Inc.,NASDAQ,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",December,2024-06-30,1.234906e+12,74774004000,24.94,1.130,...,8.11,16.48,1.212,542.81,273.82,492.57,433.53,2184730000.0,2024-06-26,2024-06-14
BRK-B,Berkshire Hathaway Inc,NYSE,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE",December,2024-03-31,9.302838e+11,107046003000,12.73,10.060,...,2.189,-,0.869,449.25,330.58,414.31,387.93,1311380000.0,None,None
LLY,Eli Lilly and Company,NYSE,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,December,2024-03-31,7.243399e+11,13373700000,117.78,1.304,...,20.82,77.19,0.409,966.1,447.24,867.06,726.83,900405000.0,2024-09-10,2024-08-15
AVGO,Broadcom Inc,NASDAQ,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,October,2024-04-30,7.044695e+11,21291000000,64.95,1.560,...,17.67,34.54,1.188,184.58,78.56,154.44,123.9,4654880000.0,2024-06-28,2024-06-24


In [16]:
data.sort_values(by='MarketCapitalization').head(10)

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
NCLH,Norwegian Cruise Line Holdings Ltd,NYSE,ENERGY & TRANSPORTATION,WATER TRANSPORTATION,December,2024-03-31,7.004214e+09,2153200000,16.42,0.311,...,2.184,9.23,2.697,21.73,12.71,17.99,17.49,439687000.0,None,None
CZR,Caesars Entertainment Corporation,NASDAQ,REAL ESTATE & CONSTRUCTION,HOTELS & MOTELS,December,2024-03-31,7.183762e+09,3714000000,9.39,0.824,...,2.84,9.21,2.985,59.5,31.74,36.61,41.25,216053000.0,None,None
BWA,BorgWarner Inc,NYSE,MANUFACTURING,MOTOR VEHICLE PARTS & ACCESSORIES,December,2024-03-31,7.283981e+09,1857000000,10.99,1.000,...,0.714,6.16,1.209,46.16,29.31,34.03,34.11,227838000.0,2024-06-17,2024-06-03
ETSY,Etsy Inc,NASDAQ,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",December,2024-03-31,7.493067e+09,426297000,29.26,1.109,...,3.342,23.06,2.020,97.44,55.08,61.42,68.39,116933000.0,None,None
PARA,Paramount Global Class B,NASDAQ,TECHNOLOGY,TELEVISION BROADCASTING STATIONS,December,2024-03-31,7.502204e+09,2715000000,NaN,0.291,...,0.706,23.38,1.798,17.28,9.54,11.24,12.44,625998000.0,2024-10-01,2024-09-16
IVZ,Invesco Plc,NYSE,FINANCE,INVESTMENT ADVICE,December,2024-06-30,7.569538e+09,1015800000,NaN,1.488,...,3.152,15.93,1.455,17.81,11.97,15.66,15.49,450032000.0,2024-09-04,2024-08-16
BBWI,Bath & Body Works Inc.,NYSE,TRADE & SERVICES,RETAIL-WOMEN'S CLOTHING STORES,January,2024-04-30,7.667985e+09,1568000000,8.88,0.858,...,1.654,7.47,1.834,52.76,26.9,43.09,41.33,223231000.0,2024-06-21,2024-06-07
LW,Lamb Weston Holdings Inc,NYSE,MANUFACTURING,"CANNED, FROZEN & PRESERVD FRUIT, VEG & FOOD SP...",May,2024-05-31,8.180627e+09,1411700000,11.43,1.735,...,1.849,8.72,0.732,110.32,52.67,79.34,92.62,143671000.0,2024-08-30,2024-08-02
DAY,Dayforce Inc.,NYSE,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,December,2024-03-31,8.228169e+09,238400000,157.91,5.360,...,5.57,30.72,1.268,75.53,47.08,52.83,63.16,157900000.0,None,None


In [21]:
data.groupby(['FiscalYearEnd']).agg({'Name': 'count',
                                     'MarketCapitalization': ['max', 'mean', 'min']})

Name MarketCapitalization                            
              count                  max          mean           min
FiscalYearEnd                                                       
April             4         1.045178e+11  4.069729e+10  1.271662e+10
August            7         3.624707e+11  1.086281e+11  9.219778e+09
December        369         2.254508e+12  8.736767e+10  7.004214e+09
February          2         4.619521e+10  2.946173e+10  1.272824e+10
January          16         5.465585e+11  1.115216e+11  7.667985e+09
July              5         1.928994e+11  1.058790e+11  1.381409e+10
June             22         3.036322e+12  1.830376e+11  8.715290e+09
March             9         8.176722e+10  2.970651e+10  9.923441e+09
May               9         3.523643e+11  8.175835e+10  8.180627e+09
November          4         2.460072e+11  8.426755e+10  2.117507e+10
October          11         7.044695e+11  1.216979e+11  1.382354e+10
September        23         3.450326e+12  2.159969e+11  1.040880e+10

In [28]:
data[data['FiscalYearEnd']=='September']

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
APD,Air Products and Chemicals Inc,NYSE,LIFE SCIENCES,INDUSTRIAL INORGANIC CHEMICALS,September,2024-03-31,5.525416e+10,4.115100e+09,22.45,1.372,...,4.989,13.79,0.822,299.6,209.25,263.88,257.75,222306000.0,2024-11-12,2024-10-01
AAPL,Apple Inc,NASDAQ,TECHNOLOGY,ELECTRONIC COMPUTERS,September,2024-03-31,3.450326e+12,1.296290e+11,34.94,2.351,...,9.11,26.13,1.244,237.23,163.86,207.76,187.9,15334100000.0,2024-05-16,2024-05-10
ATO,Atmos Energy Corporation,NYSE,ENERGY & TRANSPORTATION,NATURAL GAS DISTRIBUTION,September,2024-03-31,1.879022e+10,1.923928e+09,18.78,2.741,...,6.19,13.33,0.665,125.38,98.87,117.12,114.88,150877000.0,2024-06-10,2024-05-24
BDX,Becton Dickinson and Company,NYSE,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,September,2024-03-31,6.859268e+10,5.070000e+09,50.71,1.338,...,4.251,19.35,0.444,281.1,218.75,232.65,239.16,289006000.0,2024-06-28,2024-09-09
COR,Cencora Inc.,NYSE,TRADE & SERVICES,"WHOLESALE-DRUGS, PROPRIETARIES & DRUGGISTS' SU...",September,2024-03-31,4.472454e+10,3.939255e+09,24.88,1.577,...,-,-,0.453,246.19,170.45,226.51,219.91,196929000.0,2024-05-24,2024-05-09
DHI,DR Horton Inc,NYSE,REAL ESTATE & CONSTRUCTION,OPERATIVE BUILDERS,September,2024-06-30,5.768952e+10,6.516900e+09,11.91,0.599,...,1.457,8.53,1.711,180.15,99.43,148.54,142.41,326040000.0,2024-08-08,2024-08-01
EMR,Emerson Electric Company,NYSE,MANUFACTURING,ELECTRONIC & OTHER ELECTRICAL EQUIPMENT (NO CO...,September,2024-03-31,6.684416e+10,4.658000e+09,34.77,2.304,...,4.639,18.77,1.316,119.53,81.82,111.41,102.97,572100000.0,2024-06-10,2024-05-16
FFIV,F5 Networks Inc,NASDAQ,TECHNOLOGY,COMPUTER COMMUNICATIONS EQUIPMENT,September,2024-03-31,1.040880e+10,7.601650e+08,21.32,1.733,...,3.745,14.48,1.081,199.49,145.45,171.51,174.71,58611000.0,None,None
FICO,Fair Isaac Corporation,NYSE,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",September,2024-03-31,3.925870e+10,7.009520e+08,83.35,2.104,...,20.02,45.9,1.336,1658.03,810.26,1458.44,1232.63,24710900.0,2017-03-17,2017-03-01


### EBITDA

In [11]:
data['EBITDA'].describe()

count    4.320000e+02
mean     6.809880e+09
std      1.494066e+10
min     -4.107000e+09
25%      1.566658e+09
50%      2.859500e+09
75%      6.025000e+09
max      1.296290e+11
Name: EBITDA, dtype: float64

In [12]:
data[data['EBITDA']<0]

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
ALB,Albemarle Corp,NYSE,LIFE SCIENCES,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...",December,2024-03-31,1.099700e+10,-3.170970e+08,33.9,1.508,...,1.851,22.45,1.575,213.26,90.02,110.11,123.88,117527000.0,2024-10-01,2024-09-13
MRNA,Moderna Inc,NASDAQ,LIFE SCIENCES,"BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)",December,2024-06-30,3.328101e+10,-4.107000e+09,NaN,0.000,...,5.22,4.641,1.662,170.47,62.55,131.38,105.61,384396000.0,None,None


In [15]:
ebitda_cat = data['EBITDA']
bins = pd.qcut(ebitda_cat, 4)
results = (pd.Series(ebitda_cat).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,EBITDA,count,min,max
0,"(-4107000064.001, 1566657500.0]",108,-4.107000e+09,1.562630e+09
1,"(1566657500.0, 2859500000.0]",108,1.568000e+09,2.858000e+09
2,"(2859500000.0, 6025000000.0]",108,2.861000e+09,5.989000e+09
3,"(6025000000.0, 129629004000.0]",108,6.133000e+09,1.296290e+11


In [16]:
cat['ebitda_cat_bins'] = bins
cat.groupby(['Sector', 'ebitda_cat_bins']).count()['Name'].to_frame()

Name
Sector                     ebitda_cat_bins                      
ENERGY & TRANSPORTATION    (-4107000064.001, 1566657500.0]     1
                           (1566657500.0, 2859500000.0]       13
                           (2859500000.0, 6025000000.0]       22
                           (6025000000.0, 129629004000.0]     28
FINANCE                    (-4107000064.001, 1566657500.0]     6
                           (1566657500.0, 2859500000.0]        7
                           (2859500000.0, 6025000000.0]       11
                           (6025000000.0, 129629004000.0]     11
LIFE SCIENCES              (-4107000064.001, 1566657500.0]    21
                           (1566657500.0, 2859500000.0]       21
                           (2859500000.0, 6025000000.0]       16
                           (6025000000.0, 129629004000.0]     16
MANUFACTURING              (-4107000064.001, 1566657500.0]    29
                           (1566657500.0, 2859500000.0]       20
                           (2859500000.0, 6025000000.0]       23
                           (6025000000.0, 129629004000.0]     20
REAL ESTATE & CONSTRUCTION (-4107000064.001, 1566657500.0]    11
                           (1566657500.0, 2859500000.0]       14
                           (2859500000.0, 6025000000.0]       11
                           (6025000000.0, 129629004000.0]      2
TECHNOLOGY                 (-4107000064.001, 1566657500.0]    25
                           (1566657500.0, 2859500000.0]       10
                           (2859500000.0, 6025000000.0]       10
                           (6025000000.0, 129629004000.0]     18
TRADE & SERVICES           (-4107000064.001, 1566657500.0]    15
                           (1566657500.0, 2859500000.0]       23
                           (2859500000.0, 6025000000.0]       15
                           (6025000000.0, 129629004000.0]     13

In [35]:
data.groupby(['Sector']).agg({'EBITDA': ['min', 'max'],
                              'Name': 'count'})

EBITDA                Name
                                     min           max count
Sector                                                      
ENERGY & TRANSPORTATION     9.464000e+08  6.768900e+10    64
FINANCE                     3.844590e+08  1.070460e+11    60
LIFE SCIENCES              -4.107000e+09  3.305500e+10    74
MANUFACTURING               1.694130e+08  2.129100e+10    92
REAL ESTATE & CONSTRUCTION  7.140900e+08  7.003500e+09    38
TECHNOLOGY                  1.451520e+08  1.296290e+11    63
TRADE & SERVICES            1.580340e+08  9.660900e+10    66

### PERatio

In [11]:
data['PERatio'].describe()

count    450.000000
mean      32.493427
std       34.453450
min        3.444000
25%       16.285000
50%       24.350000
75%       35.675000
max      474.370000
Name: PERatio, dtype: float64

In [16]:
data[data['PERatio']<15].groupby(['Sector']).agg({'Name': 'count',
                                                  'PERatio': ['max', 'mean', 'min']})

Name PERatio                  
                           count     max       mean    min
Sector                                                    
ENERGY & TRANSPORTATION       17   14.99  10.336588  3.444
FINANCE                       31   14.80  11.905161  5.670
LIFE SCIENCES                  6   14.57  11.306667  7.510
MANUFACTURING                 15   14.23  10.261333  4.970
REAL ESTATE & CONSTRUCTION     6   13.34  10.933333  9.390
TECHNOLOGY                     9   13.88  11.745556  9.790
TRADE & SERVICES              10   14.67  12.262000  8.880

In [17]:
data[data['PERatio']>15].groupby(['Sector']).agg({'Name': 'count',
                                                  'PERatio': ['max', 'mean', 'min']})

Name PERatio                  
                           count     max       mean    min
Sector                                                    
ENERGY & TRANSPORTATION       51  187.08  26.001765  15.22
FINANCE                       30   55.20  26.095000  15.03
LIFE SCIENCES                 58  216.81  43.111379  15.58
MANUFACTURING                 71  225.90  36.767465  15.71
REAL ESTATE & CONSTRUCTION    34  165.97  47.112059  16.52
TECHNOLOGY                    53  474.37  48.018679  15.32
TRADE & SERVICES              59  158.71  37.201356  15.02

### PEGRatio

In [13]:
data['PEGRatio'].describe()

count    478.000000
mean       5.230321
std       45.326822
min     -158.720000
25%        1.190750
50%        1.870000
75%        2.967500
max      844.200000
Name: PEGRatio, dtype: float64

In [15]:
data[data['PEGRatio']<1].groupby(['Sector']).agg({'Name': 'count',
                                                 'PEGRatio': ['max', 'mean', 'min']})

Name PEGRatio                    
                           count      max       mean      min
Sector                                                       
ENERGY & TRANSPORTATION       15    0.956   0.523200    0.170
FINANCE                       17    0.992 -11.739118 -158.720
LIFE SCIENCES                 11    0.888   0.423418    0.000
MANUFACTURING                 17    0.980   0.732941    0.406
REAL ESTATE & CONSTRUCTION     7    0.902  -0.093571   -4.530
TECHNOLOGY                     9    0.960   0.655000    0.277
TRADE & SERVICES               9    0.995   0.694667    0.309

In [16]:
data[data['PEGRatio']<0]

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
CINF,Cincinnati Financial Corporation,NASDAQ,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE",December,2024-06-30,1.940693e+10,2.849000e+09,9.09,-158.72,...,1.853,-,0.658,128.23,94.82,117.83,111.77,156240000.0,2024-07-15,2024-06-18
EG,Everest Group Ltd,NYSE,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE",December,2024-03-31,1.668795e+10,NaN,5.67,-50.00,...,1.22,-,0.627,413.04,337.07,382.59,380.15,43458200.0,2024-06-14,2024-05-29
MOH,Molina Healthcare Inc,NYSE,FINANCE,HOSPITAL & MEDICAL SERVICE PLANS,December,2024-06-30,2.056274e+10,1.701000e+09,19.23,-0.62,...,0.381,8.33,0.579,423.92,282.96,308.99,354.36,58600000.0,None,None
UDR,UDR Inc,NYSE,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,December,2024-06-30,1.553992e+10,9.984390e+08,102.12,-4.53,...,11.99,19.17,0.820,42.42,29.95,40.58,37.41,329824000.0,2024-07-31,2024-07-10


In [15]:
data[data['Beta']>1].groupby('Sector').count()

,Name,Exchange,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Sector,,,,,,,,,,,,,,,,,,,,,
ENERGY & TRANSPORTATION,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
FINANCE,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
LIFE SCIENCES,15,15,15,15,15,15,15,13,15,15,...,15,15,15,15,15,15,15,15,15,15
MANUFACTURING,19,19,19,19,19,19,19,18,19,19,...,19,19,19,19,19,19,19,19,19,19
REAL ESTATE & CONSTRUCTION,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
TECHNOLOGY,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22
TRADE & SERVICES,19,19,19,19,19,19,19,18,19,19,...,19,19,19,19,19,19,19,19,19,19


In [16]:
data[(data['Beta']>1) & (data['Sector']=='TECHNOLOGY')].groupby('Industry').count()

,Name,Exchange,Sector,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Industry,,,,,,,,,,,,,,,,,,,,,
AIR-COND & WARM AIR HEATG EQUIP & COMM & INDL REFRIG EQUIP,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CABLE & OTHER PAY TELEVISION SERVICES,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
COMPUTER COMMUNICATIONS EQUIPMENT,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
CONSTRUCTION MACHINERY & EQUIP,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"CONSTRUCTION, MINING & MATERIALS HANDLING MACHINERY & EQUIP",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ELECTRONIC COMPUTERS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
MISC INDUSTRIAL & COMMERCIAL MACHINERY & EQUIPMENT,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
OIL & GAS FIELD MACHINERY & EQUIPMENT,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
SERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [9]:
data[data['PEGRatio']<1].groupby('Sector').count()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Industry,Address,FiscalYearEnd,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Sector,,,,,,,,,,,,,,,,,,,,,
ENERGY & TRANSPORTATION,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
FINANCE,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
LIFE SCIENCES,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
MANUFACTURING,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
REAL ESTATE & CONSTRUCTION,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
TECHNOLOGY,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
TRADE & SERVICES,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [8]:
data[(data['PEGRatio']<1) & (data['Beta']>1)]#.groupby('Sector').count()

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
AMD,Common Stock,Advanced Micro Devices Inc,"Advanced Micro Devices, Inc. (AMD) is an Ameri...",2488.0,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,"2485 AUGUSTINE DRIVE, SANTA CLARA, CA, US",...,11.51,62.93,1.681,227.3,93.11,163.78,152.81,1616310000.0,None,1995-04-27
AIG,Common Stock,American International Group Inc,"American International Group, Inc., also known...",5272.0,NYSE,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","175 WATER STREET, NEW YORK, NY, US",...,1.49,-,1.089,80.39,55.81,76.54,71.13,663668000.0,2024-06-28,2024-06-14
APA,Common Stock,APA Corporation,APA Corporation is the holding company for Apa...,6769.0,NASDAQ,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,"2000 POST OAK BLVD, STE 100, HOUSTON, TX, US",...,2.143,3.735,3.270,44.81,26.96,29.71,33.15,371192000.0,2024-08-22,2024-07-22
APTV,Common Stock,Aptiv PLC,Aptiv plc is an auto parts company headquarter...,1521332.0,NYSE,USD,USA,MANUFACTURING,MOTOR VEHICLE PARTS & ACCESSORIES,"5 HANOVER QUAY, GRAND CANAL DOCK, DUBLIN, IE",...,1.266,10.01,1.794,113.14,65.13,75.92,80.14,272062000.0,2020-02-19,2020-02-04
BKR,Common Stock,Baker Hughes Co,Baker Hughes Company is an American internatio...,1701605.0,NASDAQ,USD,USA,TECHNOLOGY,OIL & GAS FIELD MACHINERY & EQUIPMENT,"17021 ALDINE WESTFIELD ROAD, HOUSTON, TX, US",...,1.426,9.19,1.399,36.85,27.93,33.48,32.78,997998000.0,2024-05-16,2024-05-03
BK,Common Stock,Bank of New York Mellon,"The Bank of New York Mellon Corporation, commo...",1390777.0,NYSE,USD,USA,FINANCE,STATE COMMERCIAL BANKS,"240 GREENWICH STREET, NEW YORK, NY, US",...,-,-,1.082,65.42,38.39,60.08,53.99,737957000.0,2024-08-02,2024-07-22
BBWI,Common Stock,Bath & Body Works Inc.,None,701985.0,NYSE,USD,USA,TRADE & SERVICES,RETAIL-WOMEN'S CLOTHING STORES,"THREE LIMITED PKWY, P O BOX 16000, COLUMBUS, O...",...,1.654,7.47,1.834,52.76,26.9,43.09,41.33,223231000.0,2024-06-21,2024-06-07
BLDR,Common Stock,Builders FirstSource Inc,"Builders FirstSource, Inc., manufactures and s...",1316835.0,NYSE,USD,USA,TRADE & SERVICES,RETAIL-LUMBER & OTHER BUILDING MATERIALS DEALERS,"2001 BRYAN STREET, SUITE 1600, DALLAS, TX, US",...,1.565,9.79,2.039,214.7,105.23,151.78,162.07,122057000.0,None,None
CZR,Common Stock,Caesars Entertainment Corporation,"Caesars Entertainment, Inc., formerly Eldorado...",1590895.0,NASDAQ,USD,USA,REAL ESTATE & CONSTRUCTION,HOTELS & MOTELS,"100 WEST LIBERTY STREET, SUITE 1150, RENO, NV, US",...,2.84,9.21,2.985,59.5,31.74,36.61,41.25,216053000.0,None,None


In [10]:
data[(data['PEGRatio']<1) & (data['Sector']=='FINANCE')]

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
AFL,Common Stock,Aflac Incorporated,Aflac Inc. (American Family Life Assurance Com...,4977.0,NYSE,USD,USA,FINANCE,ACCIDENT & HEALTH INSURANCE,"1932 WYNNTON RD, COLUMBUS, GA, US",...,3.017,-,0.947,95.49,70.34,88.88,83.47,568222000.0,2024-06-03,2024-05-21
AIG,Common Stock,American International Group Inc,"American International Group, Inc., also known...",5272.0,NYSE,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","175 WATER STREET, NEW YORK, NY, US",...,1.49,-,1.089,80.39,55.81,76.54,71.13,663668000.0,2024-06-28,2024-06-14
BK,Common Stock,Bank of New York Mellon,"The Bank of New York Mellon Corporation, commo...",1390777.0,NYSE,USD,USA,FINANCE,STATE COMMERCIAL BANKS,"240 GREENWICH STREET, NEW YORK, NY, US",...,-,-,1.082,65.42,38.39,60.08,53.99,737957000.0,2024-08-02,2024-07-22
CNC,Common Stock,Centene Corp,Centene Corporation is a large publicly traded...,1071739.0,NYSE,USD,USA,FINANCE,HOSPITAL & MEDICAL SERVICE PLANS,"7700 FORSYTH BLVD., ST LOUIS, MO, US",...,0.241,6.6,0.500,81.42,60.83,69.82,73.46,533656000.0,None,None
CI,Common Stock,Cigna Corp,Cigna is an American multinational managed hea...,1739940.0,NYSE,USD,USA,FINANCE,HOSPITAL & MEDICAL SERVICE PLANS,"900 COTTAGE GROVE ROAD, BLOOMFIELD, CT, US",...,0.627,-,0.525,364.21,251.86,335.55,324.17,284074000.0,2024-06-20,2024-09-04
CINF,Common Stock,Cincinnati Financial Corporation,Cincinnati Financial Corporation offers proper...,20286.0,NASDAQ,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE","6200 S GILMORE RD, FAIRFIELD, OH, US",...,1.853,-,0.658,128.23,94.82,117.83,111.77,156240000.0,2024-07-15,2024-06-18
C,Common Stock,Citigroup Inc,Citigroup Inc. is an American multinational in...,831001.0,NYSE,USD,USA,FINANCE,NATIONAL COMMERCIAL BANKS,"388 GREENWICH STREET, NEW YORK, NY, US",...,-,-,1.451,67.81,37.0,63.12,55.32,1925700000.0,2024-08-23,2024-08-05
CFG,Common Stock,"Citizens Financial Group, Inc.","Citizens Financial Group, Inc. is an American ...",759944.0,NYSE,USD,USA,FINANCE,STATE COMMERCIAL BANKS,"870 WESTMINSTER ST, 1 CITIZENS PLZ THIRD FL, P...",...,-,-,1.342,43.96,21.82,36.43,32.51,452962000.0,2024-08-14,2024-07-31


In [7]:
data[data['Industry']=='SERVICES-PREPACKAGED SOFTWARE']

,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
ADBE,Common Stock,Adobe Systems Incorporated,Adobe Inc. is an American multinational comput...,796343.0,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"345 PARK AVE, SAN JOSE, CA, US",...,10.52,29.33,1.297,638.25,433.97,512.42,547.17,443400000.0,None,2005-03-24
ANSS,Common Stock,ANSYS Inc,"Ansys, Inc. is an American company based in Ca...",1013462.0,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"2600 ANSYS DRIVE, SOUTHPOINTE, CANONSBURG, PA, US",...,12.72,40.65,1.108,364.31,258.01,324.84,321.21,87300000.0,None,None
ADSK,Common Stock,Autodesk Inc,"Autodesk, Inc. is an American multinational so...",769397.0,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"111 MCINNIS PKWY, SAN RAFAEL, CA, US",...,9.29,40.31,1.485,279.53,192.01,230.93,232.68,215509000.0,None,2005-03-22
CDNS,Common Stock,Cadence Design Systems Inc,"Cadence Design Systems, Inc., headquartered in...",813672.0,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,"2655 SEELY AVENUE BLDG 5, SAN JOSE, CA, US",...,20.29,56.7,1.033,328.99,217.77,300.3,285.82,273820000.0,None,None


In [21]:
data[data['BookValue']<0]

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
MO,Altria Group,NYSE,MANUFACTURING,CIGARETTES,December,2024-03-31,84850926000.0,12306000000.0,10.33,6.310,...,4.539,7.54,0.690,49.92,36.53,46.33,42.8,1717630000.0,2024-07-10,2024-06-14
AON,Aon PLC,NYSE,FINANCE,"INSURANCE AGENTS, BROKERS & SERVICE",December,2024-03-31,65037742000.0,4407000000,23.37,1.523,...,5.38,18.41,0.907,342.46,267.42,291.56,307.45,217431000.0,2024-08-15,2024-08-01
AZO,AutoZone Inc,NYSE,TRADE & SERVICES,RETAIL-AUTO & HOME SUPPLY STORES,August,2024-05-31,50252984000.0,4247429000,20.37,1.479,...,3.521,14.93,0.712,3256.37,2375.35,2874.06,2807.22,17082800.0,None,None
BBWI,Bath & Body Works Inc.,NYSE,TRADE & SERVICES,RETAIL-WOMEN'S CLOTHING STORES,January,2024-04-30,7667985000.0,1568000000,8.88,0.858,...,1.654,7.47,1.834,52.76,26.9,43.09,41.33,223231000.0,2024-06-21,2024-06-07
BA,The Boeing Company,NYSE,MANUFACTURING,AIRCRAFT,December,2024-03-31,110542103000.0,2675000000,NaN,6.530,...,1.807,60.73,1.562,267.54,159.7,181.71,198.02,613884000.0,2020-03-06,2020-02-13
BKNG,Booking Holdings Inc,NASDAQ,ENERGY & TRANSPORTATION,TRANSPORTATION SERVICES,December,2024-03-31,129120600000.0,6808000000,28.65,1.009,...,5.75,17.45,1.385,4144.32,2719.8,3881.8,3511.07,33927500.0,2024-06-28,2024-06-07
CAH,Cardinal Health Inc,NYSE,TRADE & SERVICES,"WHOLESALE-DRUGS, PROPRIETARIES & DRUGGISTS' SU...",June,2024-03-31,23604081000.0,2783000000,42.88,0.309,...,0.119,13.58,0.622,114.93,83.4,97.97,102.67,243567000.0,2024-07-15,2024-07-01
DPZ,Domino’s Pizza Inc,NYSE,TRADE & SERVICES,WHOLESALE-GROCERIES & RELATED PRODUCTS,December,2024-06-30,14599563000.0,909234000,25.71,2.980,...,4.989,24.04,0.875,541.2,327.02,497.03,444.1,34973200.0,2024-09-30,2024-09-13
ETSY,Etsy Inc,NASDAQ,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",December,2024-03-31,7493067000.0,426297000,29.26,1.109,...,3.342,23.06,2.020,97.44,55.08,61.42,68.39,116933000.0,None,None


In [22]:
data['BookValue'].describe()

count       215.000000
mean       1887.543758
std       27107.109012
min        -282.210000
25%          13.895000
50%          29.510000
75%          52.805000
max      397506.280000
Name: BookValue, dtype: float64

In [10]:
data['DividendPerShare'].describe()

count       239.000000
mean       1700.544423
std       25710.173677
min        -282.210000
25%          12.910000
50%          26.590000
75%          50.815000
max      397506.280000
Name: DividendPerShare, dtype: float64

In [17]:
data['DividendYield'].describe()

count    184.000000
mean       0.024757
std        0.014255
min        0.001500
25%        0.013500
50%        0.022050
75%        0.034425
max        0.079400
Name: DividendYield, dtype: float64

In [8]:
data['EPS'].describe()

count    263.000000
mean       7.391217
std       13.978241
min      -12.730000
25%        2.165000
50%        4.360000
75%        8.795000
max      144.440000
Name: EPS, dtype: float64

In [9]:
data['RevenuePerShareTTM'].describe()

count       289.000000
mean        966.700194
std       15019.554067
min           2.953000
25%          19.560000
50%          37.600000
75%          80.280000
max      255402.660000
Name: RevenuePerShareTTM, dtype: float64

In [9]:
data['ProfitMargin'].describe()

count    313.000000
mean       0.135191
std        0.143140
min       -1.162000
25%        0.062200
50%        0.131000
75%        0.196000
max        0.650000
Name: ProfitMargin, dtype: float64

In [10]:
data['ReturnOnAssetsTTM'].describe()

count    361.000000
mean       0.066311
std        0.051765
min       -0.154000
25%        0.029500
50%        0.053100
75%        0.088300
max        0.309000
Name: ReturnOnAssetsTTM, dtype: float64

In [12]:
data[data['EPS']<0][['Name', 'PERatio', 'PEGRatio', 'Beta', 'DividendPerShare', 'DividendYield']]

,Name,PERatio,PEGRatio,Beta,DividendPerShare,DividendYield
Symbol,,,,,,
MMM,3M Company,NaN,1.905,1.001,8.79,0.0267
BAX,Baxter International Inc,NaN,1.696,0.574,16.04,0.0327
BIO,Bio-Rad Laboratories Inc,NaN,1.190,0.930,317.32,NaN
BA,The Boeing Company,NaN,6.530,1.562,-27.73,NaN
BMY,Bristol-Myers Squibb Company,NaN,31.350,0.439,8.13,0.0558
CTLT,Catalent Inc,NaN,2.057,1.161,19.96,NaN
DLTR,Dollar Tree Inc,NaN,2.138,0.886,33.83,NaN
ES,Eversource Energy,NaN,2.006,0.604,41.39,0.0443
HAS,Hasbro Inc,NaN,0.780,0.607,8.41,0.0434


In [18]:
data[((data['DividendYield'] >= 0.02) | (data['DividendYield'] <= 0.06))][clmns]

,Name,Beta,DividendYield
Symbol,,,
MMM,3M Company,1.001,0.0267
AOS,Smith AO Corporation,1.141,0.0144
ABT,Abbott Laboratories,0.718,0.0212
ABBV,AbbVie Inc,0.623,0.0357
ACN,Accenture plc,1.227,0.0157
...,...,...,...
LMT,Lockheed Martin Corporation,0.455,0.0230
L,Loews Corp,0.836,0.0032
LOW,Lowe's Companies Inc,1.067,0.0191


In [31]:
data[data['BookValue']==data['BookValue'].max()][clmns]

,Name,PERatio,PEGRatio,Beta
Symbol,,,,
BRK-B,Berkshire Hathaway Inc,12.73,10.06,0.869


In [12]:
data[data['DividendPerShare']==data['DividendPerShare'].max()][clmns]

,Name,PERatio,PEGRatio,Beta
Symbol,,,,
BRK-B,Berkshire Hathaway Inc,12.73,10.06,0.869


In [23]:
data[data['DividendYield']==data['DividendYield'].max()][clmns]

,Name,PERatio,PEGRatio,Beta
Symbol,,,,
MO,Altria Group,10.33,6.31,0.69


In [19]:
data[data['RevenuePerShareTTM']==data['RevenuePerShareTTM'].max()][clmns]

,Name,Beta,DividendYield
Symbol,,,
BRK-B,Berkshire Hathaway Inc,0.869,NaN


In [20]:
data[data['ProfitMargin']==data['ProfitMargin'].max()][clmns]

,Name,Beta,DividendYield
Symbol,,,
EMR,Emerson Electric Company,1.316,0.018


In [32]:
data[data['OperatingMarginTTM']==data['OperatingMarginTTM'].min()][clmns]

,Name,PERatio,PEGRatio,Beta,DividendPerShare,DividendYield
Symbol,,,,,,
MRNA,Moderna Inc,NaN,0.0,1.662,30.5,NaN


In [17]:
data[data['OperatingMarginTTM']==data['OperatingMarginTTM'].max()][clmns]

,Name,Beta,DividendYield
Symbol,,,
MAR,Marriott International Inc,1.601,0.0118


In [31]:
data[data['ReturnOnAssetsTTM']==data['ReturnOnAssetsTTM'].min()][clmns]

,Name,PERatio,PEGRatio,Beta,DividendPerShare,DividendYield
Symbol,,,,,,
MRNA,Moderna Inc,NaN,0.0,1.662,30.5,NaN


In [28]:
betas = data[data['RevenuePerShareTTM']>data['RevenuePerShareTTM'].quantile(0.75)]['Beta']
bins = pd.qcut(betas, 4)
results = (pd.Series(betas).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,Beta,count,min,max
0,"(0.43, 0.694]",18,0.431,0.683
1,"(0.694, 0.988]",18,0.704,0.988
2,"(0.988, 1.318]",17,0.989,1.305
3,"(1.318, 2.039]",18,1.331,2.039


In [15]:
profit_margin = data['ProfitMargin']
bins = pd.qcut(profit_margin, 8)
results = (pd.Series(profit_margin).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,ProfitMargin,count,min,max
0,"(-1.1629999999999998, 0.033]",46,-1.1620,0.0330
1,"(0.033, 0.0638]",46,0.0332,0.0638
2,"(0.0638, 0.104]",44,0.0661,0.1040
3,"(0.104, 0.132]",45,0.1050,0.1320
4,"(0.132, 0.168]",45,0.1330,0.1680
5,"(0.168, 0.199]",45,0.1690,0.1990
6,"(0.199, 0.263]",48,0.2020,0.2630
7,"(0.263, 0.65]",42,0.2650,0.6500


In [16]:
cat = data.copy()

In [17]:
cat['profit_margin_bins'] = bins
cat.groupby(['Sector', 'profit_margin_bins']).count()['Name'].to_frame()

Name
Sector                     profit_margin_bins                
ENERGY & TRANSPORTATION    (-1.1629999999999998, 0.033]     2
                           (0.033, 0.0638]                 10
                           (0.0638, 0.104]                  7
                           (0.104, 0.132]                  10
                           (0.132, 0.168]                   7
                           (0.168, 0.199]                   4
                           (0.199, 0.263]                   8
                           (0.263, 0.65]                    4
FINANCE                    (-1.1629999999999998, 0.033]     6
                           (0.033, 0.0638]                  2
                           (0.0638, 0.104]                  4
                           (0.104, 0.132]                   2
                           (0.132, 0.168]                   4
                           (0.168, 0.199]                  11
                           (0.199, 0.263]                   9
                           (0.263, 0.65]                    9
LIFE SCIENCES              (-1.1629999999999998, 0.033]    10
                           (0.033, 0.0638]                  7
                           (0.0638, 0.104]                  8
                           (0.104, 0.132]                   9
                           (0.132, 0.168]                   2
                           (0.168, 0.199]                  11
                           (0.199, 0.263]                   5
                           (0.263, 0.65]                    3
MANUFACTURING              (-1.1629999999999998, 0.033]    11
                           (0.033, 0.0638]                 12
                           (0.0638, 0.104]                 13
                           (0.104, 0.132]                   9
                           (0.132, 0.168]                  11
                           (0.168, 0.199]                   2
                           (0.199, 0.263]                  12
                           (0.263, 0.65]                    5
REAL ESTATE & CONSTRUCTION (-1.1629999999999998, 0.033]     1
                           (0.033, 0.0638]                  4
                           (0.0638, 0.104]                  2
                           (0.104, 0.132]                   3
                           (0.132, 0.168]                   4
                           (0.168, 0.199]                   4
                           (0.199, 0.263]                   3
                           (0.263, 0.65]                    7
TECHNOLOGY                 (-1.1629999999999998, 0.033]     3
                           (0.033, 0.0638]                  5
                           (0.0638, 0.104]                  3
                           (0.104, 0.132]                   5
                           (0.132, 0.168]                   9
                           (0.168, 0.199]                  11
                           (0.199, 0.263]                   9
                           (0.263, 0.65]                    6
TRADE & SERVICES           (-1.1629999999999998, 0.033]    13
                           (0.033, 0.0638]                  6
                           (0.0638, 0.104]                  7
                           (0.104, 0.132]                   7
                           (0.132, 0.168]                   8
                           (0.168, 0.199]                   2
                           (0.199, 0.263]                   2
                           (0.263, 0.65]                    8

In [18]:
op_margin = data['OperatingMarginTTM']
bins = pd.qcut(op_margin, 8)
results = (pd.Series(op_margin).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,OperatingMarginTTM,count,min,max
0,"(-5.6610000000000005, 0.0693]",47,-5.660,0.0693
1,"(0.0693, 0.112]",44,0.070,0.1120
2,"(0.112, 0.157]",47,0.115,0.1570
3,"(0.157, 0.193]",43,0.158,0.1930
4,"(0.193, 0.222]",45,0.195,0.2220
5,"(0.222, 0.28]",47,0.223,0.2800
6,"(0.28, 0.338]",43,0.281,0.3380
7,"(0.338, 0.703]",45,0.341,0.7030


In [19]:
cat['op_margin_bins'] = bins
cat.groupby(['Sector', 'op_margin_bins']).count()['Name'].to_frame()

Name
Sector                     op_margin_bins                     
ENERGY & TRANSPORTATION    (-5.6610000000000005, 0.0693]     4
                           (0.0693, 0.112]                   5
                           (0.112, 0.157]                    6
                           (0.157, 0.193]                    8
                           (0.193, 0.222]                    4
                           (0.222, 0.28]                    11
                           (0.28, 0.338]                     8
                           (0.338, 0.703]                    6
FINANCE                    (-5.6610000000000005, 0.0693]     5
                           (0.0693, 0.112]                   3
                           (0.112, 0.157]                    4
                           (0.157, 0.193]                    3
                           (0.193, 0.222]                    3
                           (0.222, 0.28]                     7
                           (0.28, 0.338]                     8
                           (0.338, 0.703]                   14
LIFE SCIENCES              (-5.6610000000000005, 0.0693]     5
                           (0.0693, 0.112]                   6
                           (0.112, 0.157]                   13
                           (0.157, 0.193]                    6
                           (0.193, 0.222]                    8
                           (0.222, 0.28]                     8
                           (0.28, 0.338]                     6
                           (0.338, 0.703]                    3
MANUFACTURING              (-5.6610000000000005, 0.0693]    14
                           (0.0693, 0.112]                  13
                           (0.112, 0.157]                   10
                           (0.157, 0.193]                   10
                           (0.193, 0.222]                   10
                           (0.222, 0.28]                     8
                           (0.28, 0.338]                     7
                           (0.338, 0.703]                    3
REAL ESTATE & CONSTRUCTION (-5.6610000000000005, 0.0693]     2
                           (0.0693, 0.112]                   1
                           (0.112, 0.157]                    2
                           (0.157, 0.193]                    5
                           (0.193, 0.222]                    4
                           (0.222, 0.28]                     0
                           (0.28, 0.338]                     9
                           (0.338, 0.703]                    5
TECHNOLOGY                 (-5.6610000000000005, 0.0693]     2
                           (0.0693, 0.112]                  10
                           (0.112, 0.157]                    4
                           (0.157, 0.193]                    3
                           (0.193, 0.222]                   11
                           (0.222, 0.28]                    10
                           (0.28, 0.338]                     5
                           (0.338, 0.703]                    6
TRADE & SERVICES           (-5.6610000000000005, 0.0693]    15
                           (0.0693, 0.112]                   6
                           (0.112, 0.157]                    8
                           (0.157, 0.193]                    8
                           (0.193, 0.222]                    5
                           (0.222, 0.28]                     3
                           (0.28, 0.338]                     0
                           (0.338, 0.703]                    8

In [21]:
roa = data['ReturnOnAssetsTTM']
bins = pd.qcut(op_margin, 8)
results = (pd.Series(roa).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,OperatingMarginTTM,count,min,max
0,"(-5.6610000000000005, 0.0693]",47,-0.1540,0.0838
1,"(0.0693, 0.112]",44,-0.0157,0.1200
2,"(0.112, 0.157]",47,0.0092,0.1700
3,"(0.157, 0.193]",43,0.0082,0.3090
4,"(0.193, 0.222]",45,0.0093,0.2260
5,"(0.222, 0.28]",47,0.0035,0.1710
6,"(0.28, 0.338]",43,0.0070,0.2260
7,"(0.338, 0.703]",45,0.0089,0.2690


In [28]:
cat['roa_bins'] = bins
cat.groupby(['Sector', 'roa_bins']).count()['Name'].to_frame()

Name
Sector                     roa_bins                           
ENERGY & TRANSPORTATION    (-5.6610000000000005, 0.0693]     4
                           (0.0693, 0.112]                   5
                           (0.112, 0.157]                    6
                           (0.157, 0.193]                    8
                           (0.193, 0.222]                    4
                           (0.222, 0.28]                    11
                           (0.28, 0.338]                     8
                           (0.338, 0.703]                    6
FINANCE                    (-5.6610000000000005, 0.0693]     5
                           (0.0693, 0.112]                   3
                           (0.112, 0.157]                    4
                           (0.157, 0.193]                    3
                           (0.193, 0.222]                    3
                           (0.222, 0.28]                     7
                           (0.28, 0.338]                     8
                           (0.338, 0.703]                   14
LIFE SCIENCES              (-5.6610000000000005, 0.0693]     5
                           (0.0693, 0.112]                   6
                           (0.112, 0.157]                   13
                           (0.157, 0.193]                    6
                           (0.193, 0.222]                    8
                           (0.222, 0.28]                     8
                           (0.28, 0.338]                     6
                           (0.338, 0.703]                    3
MANUFACTURING              (-5.6610000000000005, 0.0693]    14
                           (0.0693, 0.112]                  13
                           (0.112, 0.157]                   10
                           (0.157, 0.193]                   10
                           (0.193, 0.222]                   10
                           (0.222, 0.28]                     8
                           (0.28, 0.338]                     7
                           (0.338, 0.703]                    3
REAL ESTATE & CONSTRUCTION (-5.6610000000000005, 0.0693]     2
                           (0.0693, 0.112]                   1
                           (0.112, 0.157]                    2
                           (0.157, 0.193]                    5
                           (0.193, 0.222]                    4
                           (0.222, 0.28]                     0
                           (0.28, 0.338]                     9
                           (0.338, 0.703]                    5
TECHNOLOGY                 (-5.6610000000000005, 0.0693]     2
                           (0.0693, 0.112]                  10
                           (0.112, 0.157]                    4
                           (0.157, 0.193]                    3
                           (0.193, 0.222]                   11
                           (0.222, 0.28]                    10
                           (0.28, 0.338]                     5
                           (0.338, 0.703]                    6
TRADE & SERVICES           (-5.6610000000000005, 0.0693]    15
                           (0.0693, 0.112]                   6
                           (0.112, 0.157]                    8
                           (0.157, 0.193]                    8
                           (0.193, 0.222]                    5
                           (0.222, 0.28]                     3
                           (0.28, 0.338]                     0
                           (0.338, 0.703]                    8

## Overview

In [25]:
data.groupby(['Sector']).count()['Name'].to_frame()

,Name
Sector,
ENERGY & TRANSPORTATION,59
FINANCE,53
LIFE SCIENCES,64
MANUFACTURING,84
REAL ESTATE & CONSTRUCTION,36
TECHNOLOGY,55
TRADE & SERVICES,58


## Assets Filter

In [12]:
data[  (data['PERatio']<16)
     & (data['Beta']>1)
     & (data['PEGRatio']<1)
     & (data['BookValue']>30)
     & (data['DividendPerShare']>0)
     & ((data['DividendYield'] >= 0.02) | (data['DividendYield'] <= 0.06))
     & (data['EPS']>0)
     & (data['ProfitMargin']>0.07)
     & (data['OperatingMarginTTM']>0.15)
     & (data['ReturnOnAssetsTTM']>0.05)
    ]

,Name,Exchange,Sector,Industry,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Symbol,,,,,,,,,,,,,,,,,,,,,
COP,ConocoPhillips,NYSE,ENERGY & TRANSPORTATION,PETROLEUM REFINING,December,2024-03-31,1.290754e+11,2.428700e+10,12.55,0.721,...,2.957,6.48,1.249,134.54,105.08,114.17,117.16,1164310000.0,2024-06-03,2024-05-10
DHI,DR Horton Inc,NYSE,REAL ESTATE & CONSTRUCTION,OPERATIVE BUILDERS,September,2024-06-30,5.768952e+10,6.516900e+09,11.91,0.599,...,1.457,8.53,1.711,180.15,99.43,148.54,142.41,326040000.0,2024-08-08,2024-08-01
PHM,PulteGroup Inc,NYSE,REAL ESTATE & CONSTRUCTION,OPERATIVE BUILDERS,December,2024-06-30,2.566034e+10,3.721209e+09,9.44,0.339,...,1.532,6.8,1.615,135.62,68.43,116.24,105.82,207524000.0,2024-07-02,2024-06-18


In [6]:
data.to_csv('../data/sp500_Company_Overview.csv')